## Gridradar API
https://gridradar.net/de

In [1]:
#!/usr/bin/python3

import requests
import json
import pprint
from lib.gridradar_api import *
from lib.basic_functions import *
from config import *


In [2]:
influx_host = config.influx_host

NameError: name 'config' is not defined

https://api.gridradar.net/aggr_functions?token=4W01v742EaQ9iZj11B55vjaA4RyOtsNmu6M26EH09mFxlswbOSxA2X9bxuwZOBnpaYXtXyXf+xeQYl2LnQxev35WbGQVruURZteEpe52C0FVv08wXcb9Xm5f6Dvn/L3a


##  Metrics
**KEY**                            DESCRIPTION\
**- frequency**\
PMU selection, frequency 100ms, max period per query 1h\
**- frequency-mean-1m**\
PMU selection, 1m avg., max period per query 30d\
**- frequency-mean-1s**\
PMU selection, 1s avg., max period per query 12h\
**- frequency-ucte-median-1s**\
Median of the pmu 1 second mean frequencies\
**- net-time**\
Network time deviation of the UCTE power grid\
**- phase-angle**\
PMU selection, phase angle 100ms, max period per query 1h\
**- phase-angle-first-1m**\
PMU selection, first val. of minute, max period per query 30d\
**- phase-angle-first-1s**\
PMU selection, first val. of second, max period per query 12h\


columns	
0	"function"
1	"description"
data	
0	
0	"count"
1	"number of non-null values"
1	
0	"mean"
1	"arithmetic average of values"
2	
0	"median"
1	"middle value of the sorted list of values."
3	
0	"spread"
1	"difference between the minimum and maximum values"
4	
0	"stddev"
1	"the standard deviation of values."
5	
0	"sum"
1	"sum of values"
6	
0	"difference"
1	"difference between subsequent values"
7	
0	"min"
1	"lowest value"
8	
0	"max"
1	"greatest value"
9	
0	"first"
1	"the value with the oldest timestamp"
10	
0	"last"
1	"the value with the newest timestamp"

In [2]:
#dict of available metrics
metrics_dict={
    "frequency":{"aggr":"100ms", "timerange":1},        #'"1h"'
    "frequency-mean-1m":{"aggr":"1m", "timerange":720}, #'"30d"'
    "frequency-mean-1s":{"aggr":"1s", "timerange":12},  #'"12h"'
    "frequency-ucte-median-1s":{"aggr":"1s", "timerange":24}, #'"1d"'
    "net-time":{"aggr":"1s", "timerange":24},           #'"1d"'
    "phase-angle":{"aggr":"100ms", "timerange":1},      #'"1h"'
    "phase-angle-first-1m":{"aggr":"1m", "timerange":720}, #'"30d"'
    "phase-angle-first-1m":{"aggr":"1s", "timerange":12} #'"12h"'
}
metric = "frequency"
settings = metrics_dict[metric]
aggr = settings['aggr']
timerange = settings['timerange']


## get data from API with dynamic time ranges

In [5]:
## Request and header parameters
url = 'https://api.gridradar.net/query'
token = '<token>'

headers = {
    'Content-type': 'application/json', 
    'Authorization': 'Bearer '+token
}
querypmu = 'berlin'

In [25]:
metrics_dict={
    #"frequency":{"aggr":"100ms", "timerange":1}        #'"1h"'
    "frequency-mean-1m":{"aggr":"1m", "timerange":720}
    }
for metric, settings in metrics_dict.items():
    ## define API query parameters
    request = metric_request_creator(metric,metrics_dict,querypmu)
    pprint.pprint(request)
    ## Converting the Python dictionary to a JSON string
    json_request = json.dumps(request)

    ## Request execution and response reception
    response = requests.post(url, data=json_request, headers=headers)
    pprint.pprint(response)
    ## Converting the JSON response string to a Python dictionary
    result_dict = json.loads(response.content)

    ## Pretty print response dictionary
    pprint.pprint(result_dict)

{'aggr': '1m',
 'format': 'json',
 'from': '2023-03-18T20:56:56Z',
 'metric': 'frequency-mean-1m',
 'pmus': 'berlin',
 'to': '2023-04-17T20:56:56Z',
 'ts': 'rfc3339'}
<Response [200]>
[{'datapoints': [[None, '2023-03-18 20:56:00'],
                 [50.033412, '2023-03-18 20:57:00'],
                 [50.050216, '2023-03-18 20:58:00'],
                 [50.035693, '2023-03-18 20:59:00'],
                 [49.996778, '2023-03-18 21:00:00'],
                 [49.968522, '2023-03-18 21:01:00'],
                 [49.960886, '2023-03-18 21:02:00'],
                 [49.972238, '2023-03-18 21:03:00'],
                 [49.974718, '2023-03-18 21:04:00'],
                 [49.976672, '2023-03-18 21:05:00'],
                 [49.983802, '2023-03-18 21:06:00'],
                 [50.007963, '2023-03-18 21:07:00'],
                 [49.990143, '2023-03-18 21:08:00'],
                 [49.987243, '2023-03-18 21:09:00'],
                 [49.993365, '2023-03-18 21:10:00'],
                 [50.00215

## write data to influxdb

In [23]:
#InfluxDB configuration
import influxdb
influx_host=''  # os.environ.get('INFLUX_HOST')
influx_port=8086 # os.environ.get('INFLUX_PORT')
influx_username = ''
influx_password = ''
influx_database = 'gridradar'
location='' # os.environ.get('LOCATION')

influxdb_client = influxdb.InfluxDBClient(
            influx_host,
            influx_port,
            influx_username,
            influx_password,
            influx_database, 
            ssl = False,
            verify_ssl = True
        )

In [24]:

influxdb_client.write_points(out_influxDB,database= influx_database,
                                    time_precision="ms",
                                    batch_size=10000,
                                    protocol='json')

True

In [19]:
import logging
import math
def convert_dataset(response,metric,location,ctr,uptime):
    pprint.pprint(response.status_code)
    logging.debug("API response status code: {}".format(response.status_code))
    #pprint.pprint(response.content)
    ## Converting the JSON response string to a Python dictionary
    result_dict = json.loads(response.content)
    #pprint.pprint(result_dict)
    messdaten=result_dict[0]

    outDB=[]
    for values in messdaten['datapoints']:
        #print(type(values[0]))
        if math.isnan(values[0])==False:
            outDB_new = {
                'measurement': metric,
                'tags': {'location': location},
                'fields': {'value': values[0]},
                'time':timestamp_convert(values[1])
                }
            outDB.append(outDB_new)
    # add response state and time to data output for debugging purposes
    if metric=='median_frequency':
        request_type=1
    elif metric=='net_time':
        request_type=2
    else:
        request_type=0

    debug_info = {
        'measurement': 'API_stats',
        'tags': {'location': location},
        'fields': {
            'request_counter': ctr,
            'request_type':request_type,
            'stat_code':response.status_code,
            'uptime':int(uptime)},
            'time':time_now()}
    outDB.append(debug_info)
    #print(type(outDB))
    #outDB
    return(outDB)

def data_wrangler(url,request,metric,location,headers,ctr,uptime):
    try:
        ctr=ctr+1
        ## Converting the Python dictionary to a JSON string
        json_request = json.dumps(request)
        ## Request execution and response reception
        response = requests.post(url, data=json_request, headers=headers)

        out_influxDB=convert_dataset(response,metric,location,ctr,uptime)
        #pprint.pprint(out_influxDB)
        #logging.debug("Read initial value: {:.2f}".format(m3abs))
        
        client.write_points(out_influxDB, database='gridradar', time_precision='ms', batch_size=10000, protocol='json') 
        #print('gridradar2influx.py  --  request counter: ',str(ctr),'for ',measurement,' written to InfluxDB')
        logging.info('gridradar2influx.py  --  request counter: {} for {} written to InfluxDB'.format(ctr,metric))
        #logging.info("Read initial value: {:.2f}".format(m3abs))
    except (ValueError, TypeError) as ex:
        #debug_str='"%s" cannot be converted to an float: %s' % (raw_content, ex)
        print(ex)
        
    return ctr

In [28]:
uptime=0
ctr=0
print(metric)
out_influxDB=convert_dataset(response,metric,querypmu,ctr,uptime)

frequency-mean-1m
200


TypeError: must be real number, not NoneType

## get data from influxDB

In [29]:
from influxdb import DataFrameClient

clientDF = DataFrameClient(influx_host,
                           influx_port,
                           influx_username,
                           influx_password,
                           influx_database)

In [33]:
dffreq=clientDF.query('SELECT "value" FROM "gridradar"."autogen"."frequency"')
dffreq=dffreq['frequency']#[0]
dffreq=dffreq.rename(columns={'value': 'frequency'})
dffreq['datetime']=dffreq.index
dffreq=dffreq[dffreq["datetime"]>'2023-04-01']
dffreq.head(2)



,frequency,datetime
2023-04-17 19:35:25+00:00,50.0170,2023-04-17 19:35:25+00:00
2023-04-17 19:35:25.100000+00:00,50.0165,2023-04-17 19:35:25.100000+00:00


In [34]:
dftime=clientDF.query('SELECT "value" FROM "gridradar"."autogen"."net_time"')
dftime=dftime['net_time']
dftime=dftime.rename(columns={'value': 'net_time'})

dftime['datetime']=dftime.index
print(dftime["datetime"].max() - dftime["datetime"].min())
dftime.head(2)

KeyError: 'net_time'

In [ ]:
import plotly.express as px
dfplottime=dftime[dftime["datetime"]>'2022-02-02']
fig = px.line(dfplottime,x="datetime",y="net_time", title='gridradar net_time data from influxDB')
fig.show()